In [1]:
import sys 
from pathlib import Path 
import os 

In [2]:
parent_dir = Path(os.getcwd()).resolve().parent 
sys.path.insert(0, str(parent_dir))

In [3]:
parent_dir

PosixPath('/home/jin/gpt2_from_scratch')

In [4]:
from datasets import load_from_disk
from torch.utils.data import DataLoader
from gpt2_config.config import GPT2Config
from gpt2.gpt_model import GPT2
from data.dataset import InstructDataset, collate_fn
import tiktoken

/home/jin/gpt2_from_scratch/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


parent_folder  /home/jin/gpt2_from_scratch/gpt2


In [5]:
dataset = load_from_disk('../final_dataset')
config = GPT2Config()
model = GPT2(cfg=config)

In [6]:
model

GPT2(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (attn): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_resid): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (attn): MultiHeadAttention(
        (W_query): Linear(in_features=768

In [7]:
# prepare dataset
tokenizer = tiktoken.get_encoding('gpt2')
train_ids = InstructDataset(dataset['train'], tokenizer)
val_ids = InstructDataset(dataset['val'], tokenizer)
test_ids = InstructDataset(dataset['test'], tokenizer)

In [8]:
batch_size = 5

train_loader = DataLoader(
    train_ids, 
    batch_size=batch_size,
    shuffle=True, 
    collate_fn=collate_fn, 
    drop_last=True, 
    num_workers=0
)

test_loader = DataLoader(
    dataset=test_ids, 
    batch_size=batch_size, 
    shuffle=True, 
    collate_fn=collate_fn, 
    drop_last=True, 
    num_workers=0 
)
val_loader = DataLoader(
    dataset=val_ids, 
    batch_size=batch_size, 
    collate_fn=collate_fn, 
    shuffle=True, 
    drop_last=True, 
    num_workers=0
)

In [9]:
import torch 


In [10]:
device =  'cpu'
device

'cpu'

In [11]:
optimizer = torch.optim.AdamW(model.to(device).parameters(), lr=5e-5, weight_decay=0.1)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)

In [12]:
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: True
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 5e-05
    maximize: False
    weight_decay: 0.1
)

In [13]:
from trainer import train_model

In [ ]:
train_model(model, train_loader, val_loader, optimizer, device, num_epochs=1, eval_freq=3, eval_iter=2, start_context=4, tokenizer=tokenizer)

epoch 1 (Step 000000): Train loss 10.327, Vall loss 10.252
epoch 1 (Step 000003): Train loss 8.906, Vall loss 9.120
epoch 1 (Step 000006): Train loss 7.827, Vall loss 8.856
epoch 1 (Step 000009): Train loss 8.831, Vall loss 8.854
epoch 1 (Step 000012): Train loss 8.276, Vall loss 8.190
